# Notebook 3: Texture classification

An important topic in computer vision is to classify textures based only on photos. Many different algorithms exist for this pretty hard machine learning problem. 

In this notebook, we'll try and use persistent homology to classify textures. This exercise was inspired by [this](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w50/Chung_Smooth_Summaries_of_Persistence_Diagrams_and_Texture_Classification_CVPRW_2020_paper.pdf) recent paper (by people here at UNCG!), which is actually very readable so go check it out if you're interested. That paper uses more advanced techniques that what we'll use here, but the basic idea comes through. The textures we will use are from the classic Brodatz texture dataset. 

This Notebook has **three compulsory Exercises**. 

First let's install gudhi and an optimal transport library we'll need later too.

In [ ]:
!pip install gudhi

In [ ]:
!pip install pot

Make sure you still have the texture data from last time in your home directory.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import gudhi
import gudhi.wasserstein
import gudhi.hera
import numpy as np
import ot

Here is a function that uses everything we saw in Notebook 2. It takes in an array representing an image, and outputs the persistence.

In [ ]:
def image_persistence(array, dimension):
  height, width = array.shape
  cubeComplex = gudhi.CubicalComplex(
      dimensions = [width,height],
      top_dimensional_cells = 255 - array.flatten()
  )
  cubeComplex.compute_persistence()
  persistence = cubeComplex.persistence_intervals_in_dimension(dimension)
  return persistence

## Bottleneck distance with Gudhi
Let's learn how to compute bottleneck distance with `gudhi`. As an example, we'll take the top left corners of the `sandhe.tiff` and `barkhe.tiff` images and compare them.

In [ ]:
#sand
sand_image = Image.open("./tda-texture-exercise/sandhe.tiff")
sand_array = np.array(sand_image)
top_left_corner_of_sand =  sand_array[0:100,0:100]
plt.imshow(top_left_corner_of_sand, cmap='gray')
plt.show()

#bark
bark_image = Image.open("./tda-texture-exercise/barkhe.tiff")
bark_array = np.array(bark_image)
top_left_corner_of_bark =  bark_array[0:100,0:100]
plt.imshow(top_left_corner_of_bark, cmap='gray')
plt.show()

We can use our function above to get their persistence diagrams.

In [ ]:
#sand
sand_persistence = image_persistence(top_left_corner_of_sand, dimension=0)
gudhi.plot_persistence_diagram(sand_persistence)

#bark
bark_persistence = image_persistence(top_left_corner_of_bark, dimension=0)
gudhi.plot_persistence_diagram(bark_persistence)

We're going to use the function `gudhi.bottleneck_distance`. 

In [ ]:
gudhi.bottleneck_distance(
    sand_persistence, bark_persistence
)

Let's compare this distance to the distance between the top left corner of `sandhe.png` and the bottom right corner of the same image. Should it be higher or lower than the distance between sand and bark? 

In [ ]:
#get the bottom right corner
sand_image = Image.open("./tda-texture-exercise/sandhe.tiff")
sand_array = np.array(sand_image)
bottom_right_corner_of_sand =  sand_array[400:500,400:500]
#get the persistence
bottom_right_sand_persistence = image_persistence(bottom_right_corner_of_sand, dimension=0)
#get the bottleneck distance to the top left corner of sand
gudhi.bottleneck_distance(
    sand_persistence,
    bottom_right_sand_persistence
)

Is this what you were expecting? What does this tell you about the bottleneck distance as a measure?

## Exercise 1: Matching corners with bottleneck distance
Can persistence match top-left 100x100 corners to bottom-right 100x100 corners? **Fill in the ... below** to write code to find out.

In [ ]:
#let's record the image names
image_names = [
  'sandhe.tiff', 'grasshe.tiff', 'brickhe.tiff', 'weavehe.tiff','woodhe.tiff','bubbleshe.tiff','barkhe.tiff'
]

In [ ]:
#let's collect the top left and bottom right corners of our images
#into two lists named top_left_corner_images and bottom_right_corner_images
top_left_corner_images = []
bottom_right_corner_images = []
for image_name in image_names:
  image = Image.open("./tda-texture-exercise/{}".format(image_name))
  image_array = np.array(image)
  top_left_corner_image = image_array[...,...]
  top_left_corner_images.append(top_left_corner_image)
  bottom_right_corner_image = image_array[...,...]
  bottom_right_corner_images.append(bottom_right_corner_image)

In [ ]:
#okay now let's match each top left corner to it's closest match from 
#the bottom right corners
for n in range(len(image_names)):
  #get the persistence of the nth top left corner
  top_left_persistence = image_persistence(...)
  #get the bottleneck distance to all the bottom right corners
  distances_to_bottom_right_corners = []
  for m in range(len(image_names)):
    bottom_right_persistence = image_persistence(...)
    distance = gudhi.bottleneck_distance(
      ...,
      ...
    )
    distances_to_bottom_right_corners.append(distance)
  #now we have a list of distances, let's find the minimum one
  best_match = np.argmin(distances_to_bottom_right_corners)
  #and print the results
  print("The top left corner of the image {}".format(image_names[n]))
  print("was matched to the bottom right corner of the image {}.".format(image_names[best_match]))
  

How did we do? Did all the top-left corners find their bottom-right partners?

## Wasserstein distances
Bottleneck distance is often criticized (including in this class!) for being too insensitive. Let's try this again, but with a p-Wasserstein distance instead of a bottleneck distance. Recall that the p-Wasserstein distance is a version of bottleneck distance that replaces the maximums with vector norms. 

`gudhi` has at least one decent implementation of p-Wasserstein distances, which we'll show now. We still have `sand_persistence` and `bark_persistence` as variables (hopefully -- if you don't just run the first few blocks of code again), so we'll use those.

The `order` and `internal_p` refer to the exponents in the Wasserstein distance. We'll set both to 1 to get the 1-Wasserstein distance.

In [ ]:
gudhi.hera.wasserstein_distance(
    sand_persistence,
    bark_persistence,
    order = 1,
    internal_p = 1
)

Remember, the Wasserstein distance cannot be directly compared to bottleneck, since it's a sum not a max. 

Let's look at the two corners of sand again. Do we see a better result than with bottleneck distance?

In [ ]:
#get the bottom right corner
sand_image = Image.open("./tda-texture-exercise/sandhe.tiff")
sand_array = np.array(sand_image)
bottom_right_corner_of_sand =  sand_array[400:500,400:500]
#get the persistence
bottom_right_sand_persistence = image_persistence(bottom_right_corner_of_sand, dimension=0)
#get the bottleneck distance to the top left corner of sand
gudhi.hera.wasserstein_distance(
    sand_persistence,
    bottom_right_sand_persistence,
    order = 1,
    internal_p = 1
)

## Exercise 2: Using Wasserstein distance
In the code blocks below, **repeat Question 1 except this time use 1-Wasserstein distance**. Does it do better than the bottleneck distance? Or does it do better for some images and not in others?

## Exercise 3: Using Wasserstein distance and 1 dimensional homology
In the code blocks below, **repeat Exercise 2 except this time use 1 dimensional persistent homology**. How do the results compare?

# Making a training set for the challenge questions

Instead of taking just two pieces of every image (the top-left and bottom-right corners), we could take more. Let's say we take 5 100x100 patches of each image. That gives us a dataset of 35 images with labels. Let's call this the 'training set'.

In [ ]:
training_set = []
for image_name in image_names:
    full_image = Image.open("./tda-texture-exercise/{}".format(image_name))
    full_image_array = np.array(full_image)
    for i in range(5):
        training_set.append(full_image_array[100*i:100*(i+1), 100*i:100*(i+1)])

Let's also make some labels which correspond to this training set.

In [ ]:
training_labels = []
for image_name in image_names:
    training_labels.extend([image_name]*5)

In [ ]:
training_labels

# Challenge questions

- Create a distance matrix $M$ such that the $(i,j)^{th}$ entry of $M$ is the Wasserstein distance between the diagrams for image $i$ and image $j$ in `training_set`. Give this matrix to the clustering algorithm DBSCAN (see [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) for details and an example) and see if it correctly finds the 7 clusters. Remember to use `metric=precomputed` in your DBSCAN function.

- Build a function `classify_texture` that takes any image and returns the label of the closest match in the training set (by Wasserstein distance). You should be able to adapt some of the code above. Try your function out on the bottom _left_ corners and see if it finds the right labels.

- For each texture, generate the persistence landscapes for the five examples of that texture in the training set and plot them on top of each other. Do the persistence landscapes for different textures look different?


# ML Challenge questions

If you already like and know machine learning, then you've probably noticed you have a huge number of ways to use the above to build a texture classifier. Feel free to try some of them out, and you can even reuse what you create in your mini-project later on. Possible ideas include:
- nearest neighbor classifiers using Wasserstein distance
- support vector machines or other vector classifiers using the persistence landscapes of each image
- k-means clustering on persistence landscapes